In [ ]:
# Import libraries
import pandas as pd
import numpy as np

from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans
from surprise import accuracy
from surprise import dump
from surprise.model_selection import cross_validate
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import train_test_split

from utilities import data_basic_utility as databasic

Error: Session cannot generate requests

## File Details

Basic run with SVD without cross validation. This can be the basis for parameter tuning and other stuff later


# Assignment 3 -- Recommendation Systems

* The final challenge is much like Assignment 2 -- but scoped appropriately for the time and your current abilities.
* It is ratings prediction, just like the movielens recommendations we have seen and many other similar problems.
* The features created are based on Beer Reviews from experts on a website.
* Each beer has been scored between 0 and 5 (on a real scale, so 2.75 or 3.5 is OK).
* The official measure is Mean Average Error (MAE) which is pretty intuitive to work with. Everything supports is and it is easy to interpret.
* A set of features have been created based on the reviewer, the written review, and information about the Beer being reviewed.
* Not all features have to be used, and you can easily create new features using the data if you like.
* The features included are:

![title](Images/A3Features.png)

* Sizes of the files are:
|Size | File|
|---|---|
| 1.9G | features.tsv |
| 88B  | header-features.tsv|
| 48B  | header.tsv |
| 15M  | test.tsv |
| 50M  | train.tsv |
| 16M |  val.tsv |


In [ ]:
filePrefix = "A3_074_surprise_knnmeans"
baseDataDir = "C:/Development/Data/COSC2670/Assignment3/A3data/"
subrunDir = "subruns/"
modelsDir = "models/"
writeSubRunFile = True
seed = databasic.get_random_seed()

In [ ]:
# RowID  BeerID  ReviewerID  BeerName  BeerType  Label
#df_train = pd.read_csv(baseDataDir + 'train_wk12.tsv',sep='\t',
df_train = pd.read_csv(baseDataDir + 'train_200k.tsv',sep='\t',
                         names=['RowID','BeerID','ReviewerID',
                                  'BeerName','BeerType','rating'])
df_train.head(10)

#df_vali = pd.read_csv(baseDataDir + 'val_wk12.tsv',sep='\t',
df_vali = pd.read_csv(baseDataDir + 'vali_200k.tsv',sep='\t',
                         names=['RowID','BeerID','ReviewerID',
                                  'BeerName','BeerType','rating'])
df_vali.head(10)


,RowID,BeerID,ReviewerID,BeerName,BeerType,rating
0,22,12300,2634,Rauch �r Bock,Rauchbier,4.0
1,27,12300,5634,Rauch �r Bock,Rauchbier,4.5
2,28,12300,3544,Rauch �r Bock,Rauchbier,4.5
3,40,12300,6521,Rauch �r Bock,Rauchbier,4.0
4,43,12300,10177,Rauch �r Bock,Rauchbier,4.5
5,48,12300,2907,Rauch �r Bock,Rauchbier,3.5
6,49,12300,1532,Rauch �r Bock,Rauchbier,4.0
7,50,12300,3452,Rauch �r Bock,Rauchbier,3.5
8,59,12300,6861,Rauch �r Bock,Rauchbier,4.0
9,64,6699,6401,Caldera Pale Ale,American Pale Ale (APA),4.5


Column List: 
RowID BrewerID ABV DayofWeek Month DayofMonth Year TimeOfDay Gender Birthday Text Lemmatized POS_Tag

Collab Filtering with Surprise, doesn't use the features at all

In [ ]:
# df_features = pd.read_csv(baseDataDir + 'features-top500.tsv',sep='\t', names=['RowID','BrewerID','ABV','DayofWeek','Month',
# df_features = pd.read_csv(baseDataDir + 'features_200k.tsv',sep='\t', names=['RowID','BrewerID','ABV','DayofWeek','Month',
#                                                                  'DayofMonth','Year','TimeOfDay','Gender',
#                                                                  'Birthday','Text','Lemmatized','POS_Tag'])

# df_features.head(10)

In [ ]:
idCols = ['RowID','BeerID','ReviewerID']

# Setup the data to be just the Reviewer and the Beer(Item) and the Rating Label we want to learn.
dfTrainFeatures = df_train.drop(['RowID','BeerName','BeerType'],axis=1)
dfValiIds = df_vali[idCols]
dfValiFeatures = df_vali.drop(['RowID','BeerName','BeerType'],axis=1)

dfTrainFeatures.head()

,BeerID,ReviewerID,rating
0,12300,10635,4.0
1,12300,6547,4.5
2,12300,9789,4.5
3,12300,7372,5.0
4,12300,1302,4.5


In [ ]:

reader = Reader(rating_scale=(0, 5))
dsetTrainFeatures = Dataset.load_from_df(dfTrainFeatures[['BeerID','ReviewerID',
                                    'rating']],reader)

dsetValiFeatures = Dataset.load_from_df(dfValiFeatures[['BeerID','ReviewerID',
                                     'rating']],reader)
trainsetTrainFeatures = dsetTrainFeatures.build_full_trainset()

print(type(dsetTrainFeatures))
print(type(trainsetTrainFeatures))
trainsetTrainFeatures

<class 'surprise.dataset.DatasetAutoFolds'>
<class 'surprise.trainset.Trainset'>


In [ ]:

NA,valset = train_test_split(dsetValiFeatures, test_size=1.0)

# Medium Tuning best params: {'k': 60, 'min_k': 2, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}

# algorithm = KNNWithMeans()
algorithm = KNNWithMeans(k=80)
# algorithm = KNNWithMeans(k=60, min_k = 2, sim_options= {'name': 'msd', 'min_support': 1, 'user_based': False})

model = algorithm.fit(trainsetTrainFeatures)

# Save the predictor model to file
dump.dump(modelsDir + filePrefix + "_predictor.model", None, model, True)

predictions = algorithm.test(valset)


# Score our predictions with MAE
# It is around 0.77, which means the a random guess based on the distribution of the data
# is on average within 0.77 (plus or minus) the true label.
# Not bad! You can beat it though, I'm sure :).
# Smaller MAE is the better. Good luck!
mae = accuracy.mae(predictions,verbose=True)

print("Average MAE: " + str(mae))

Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  0.4377
Average MAE: 0.4377105335852574


In [ ]:
print(type(valset))
print(valset[0:10])


<class 'list'>
[(6056, 171, 4.0), (1103, 6197, 4.5), (10439, 3553, 2.5), (10277, 8711, 4.0), (10770, 8623, 3.5), (1324, 4886, 3.5), (11290, 4833, 2.0), (6695, 6740, 4.0), (117, 8033, 3.5), (1198, 9601, 4.0)]


In [ ]:
print(type(predictions))
print(str(len(predictions)))
print(predictions[0:10])
print(predictions[0])

<class 'list'>
39509
[Prediction(uid=6056, iid=171, r_ui=4.0, est=4.263987337640556, details={'actual_k': 56, 'was_impossible': False}), Prediction(uid=1103, iid=6197, r_ui=4.5, est=4.0808078536104535, details={'actual_k': 72, 'was_impossible': False}), Prediction(uid=10439, iid=3553, r_ui=2.5, est=3.8194091413381788, details={'actual_k': 60, 'was_impossible': False}), Prediction(uid=10277, iid=8711, r_ui=4.0, est=3.8990941892225, details={'actual_k': 62, 'was_impossible': False}), Prediction(uid=10770, iid=8623, r_ui=3.5, est=3.977155282528007, details={'actual_k': 8, 'was_impossible': False}), Prediction(uid=1324, iid=4886, r_ui=3.5, est=3.3670585815347582, details={'actual_k': 21, 'was_impossible': False}), Prediction(uid=11290, iid=4833, r_ui=2.0, est=3.2285115123834776, details={'actual_k': 55, 'was_impossible': False}), Prediction(uid=6695, iid=6740, r_ui=4.0, est=3.8523285787442756, details={'actual_k': 47, 'was_impossible': False}), Prediction(uid=117, iid=8033, r_ui=3.5, est=3

In [ ]:
# Convert the Predictions to a dataframe so we can lookup predictions easy
lstUIds = list(map(lambda x: x.uid, predictions))
lstIIds = list(map(lambda x: x.iid, predictions))
lstTrueRatings = list(map(lambda x: x.r_ui, predictions))
lstRatingEst = list(map(lambda x: x.est, predictions))


# uid == BeerId, iid == ReviewerId, r_ui == Original Ration, est = Predicted rating
dfPredictions = pd.DataFrame({ "uid": lstUIds,"iid": lstIIds, "r_ui": lstTrueRatings, "Predict": lstRatingEst })

dfPredictions.head()
# dfPredictions[dfPredictions.uid == 3519]

,uid,iid,r_ui,Predict
0,6056,171,4.0,4.263987
1,1103,6197,4.5,4.080808
2,10439,3553,2.5,3.819409
3,10277,8711,4.0,3.899094
4,10770,8623,3.5,3.977155


In [ ]:
print(dfValiIds.shape)
print(dfPredictions.shape)

(39509, 3)
(39509, 4)


In [ ]:
# join the predictions to the ids, sort by rowid and write to file
dfPredictions = pd.merge(dfValiIds, dfPredictions, how="inner", left_on=["BeerID", "ReviewerID"], right_on=["uid", "iid"])
dfPredictions.head()


,RowID,BeerID,ReviewerID,uid,iid,r_ui,Predict
0,22,12300,2634,12300,2634,4.0,4.411133
1,27,12300,5634,12300,5634,4.5,4.246060
2,28,12300,3544,12300,3544,4.5,4.432695
3,40,12300,6521,12300,6521,4.0,4.259541
4,43,12300,10177,12300,10177,4.5,4.100380


This basic normal run doesn't give good MAE, so not worth writing out and considering

In [ ]:
if writeSubRunFile:
  dfPredictions.sort_values("RowID")[["RowID", "BeerID", "ReviewerID", "Predict"]].to_csv(subrunDir + filePrefix + "_subrun.csv", index=False)

print("Average MAE: " + str(mae))
print("analyse_maes.append(" + str(mae) + ")")
print(dfPredictions.shape)
dfPredictions.sort_values("RowID").head(8)


Average MAE: 0.4377105335852574
analyse_maes.append(0.4377105335852574)
(39703, 7)


,RowID,BeerID,ReviewerID,uid,iid,r_ui,Predict
0,22,12300,2634,12300,2634,4.0,4.411133
1,27,12300,5634,12300,5634,4.5,4.246060
2,28,12300,3544,12300,3544,4.5,4.432695
3,40,12300,6521,12300,6521,4.0,4.259541
4,43,12300,10177,12300,10177,4.5,4.100380
5,48,12300,2907,12300,2907,3.5,4.064850
6,49,12300,1532,12300,1532,4.0,4.285378
7,50,12300,3452,12300,3452,3.5,4.123385


In [ ]:

print("Run - " + filePrefix)
# Log of Results
analyse_maes = []

print("Average MAE over all tests: " + str(np.mean(analyse_maes)))

# Make sure it's predicting floats
# dfPredicted["Predict"].drop_duplicates()

Run - A3_074_surprise_knnmeans
Average MAE over all tests: nan


C:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Results

* Average MAE with simple KNNWithMeans: 0.4378460143896427
* MAE with simple tuned best params (k=60): 0.43772849327628577
* MAE with simple tuned best params (k=80): 0.4377105335852574
* MAE with medium tuned best params: 0.4527693481239263
* MAE with full best params: 

